In [10]:
# ! pip install requests --upgrade

In [11]:
# ! pip install tqdm

In [1]:
import requests
import os
from tqdm import tqdm
import time
import tempfile

# download folder
d_folder = r'I:\Research_Data\Wild_fire\WRF_Chem'
os.chdir(d_folder)
os.makedirs(os.path.join(d_folder, '202306'), exist_ok=True)
os.makedirs(os.path.join(d_folder, '202307'), exist_ok=True)
os.makedirs(os.path.join(d_folder, '202308'), exist_ok=True)
os.makedirs(os.path.join(d_folder, '202309'), exist_ok=True)

In [4]:
from datetime import datetime, timedelta

def generate_dates_hourly(start_date_str, end_date_str):
    # Convert the date strings to datetime objects
    start_date = datetime.strptime(start_date_str, '%Y-%m-%d %H:%M:%S')
    end_date = datetime.strptime(end_date_str, '%Y-%m-%d %H:%M:%S')
    
    # Initialize an empty list to store the date strings
    date_list = []
    
    # Loop from the start date to the end date with a one-hour interval
    current_date = start_date
    while current_date <= end_date:
        date_list.append(current_date.strftime('%Y-%m-%d_%H:%M:%S'))
        current_date += timedelta(hours=1)
    
    return date_list

# Example usage:
start_date = "2023-06-01 00:00:00" 
end_date =   "2023-09-01 00:00:00"
hour_range = generate_dates_hourly(start_date, end_date)
print(f"Hour count: {len(hour_range)}. From: {hour_range[0]} to {hour_range[-1]}")

Hour count: 2209. From: 2023-06-01_00:00:00 to 2023-09-01_00:00:00


In [5]:
# download the data file(s)

def safe_file_write(file_path, content):
    # Create a temporary file in the same directory as the target file
    dir_name = os.path.dirname(file_path)
    with tempfile.NamedTemporaryFile(mode='wb', dir=dir_name, delete=False) as temp_file:
        try:
            # Write the content to the temporary file
            temp_file.write(content)
            temp_file.flush()
            os.fsync(temp_file.fileno())
            temp_file.close()

            # Rename the temporary file to the target file (atomic operation)
            os.replace(temp_file.name, file_path)
        except Exception as e:
            # Handle the exception (e.g., log it, raise it, or silently ignore it)
            print(f"An error occurred while writing the file: {e}")
            os.unlink(temp_file.name)
        else:
            print("File successfully written.")

def download_all_files(hour_range):
    total_cnt = len(hour_range)
    while len(hour_range) > 0:
        downloaded_cnt = total_cnt - len(hour_range)
        # print(f"Downloaded {downloaded_cnt} / {total_cnt} hours.")

        hour = hour_range.pop()
        try:
            
            year = hour[:4] 
            month = hour[5:7]
            hms = hour[-11:]

            url = f"https://data.rda.ucar.edu/ds340.0/{year}{month}/wrfout_hourly_d01_{year}-{month}-{hms}.nc"  
            # e.g.: https://data.rda.ucar.edu/ds340.0/202306/wrfout_hourly_d01_2023-06-01_00:00:00.nc
  
            basename = url.split(r'/')[-1].replace(":", "_")
            filename = os.path.join(d_folder, f"{year}{month}", basename)

             
            # print("Downloading url: ", url)
            print(url)
            if os.path.exists(filename):
                print(f"File {filename} exists, skipped!")
                continue

            response = requests.get(url)

            with open(filename, "wb") as f:
                f.write(response.content)

            safe_file_write(file_path=filename, content=response.content) 
            

            print("Saved at:", filename)

        except Exception as e:
            hour_range.append(hour)
            print("Error when downloading:", url)
            print("Error information:", e)
            print("sleeping 10 seconds...")
            time.sleep(10)

            download_all_files(hour_range=hour_range)

        continue
            
download_all_files(hour_range=hour_range)

https://data.rda.ucar.edu/ds340.0/202309/wrfout_hourly_d01_2023-09-01_00:00:00.nc
File successfully written.
Saved at: I:\Research_Data\Wild_fire\WRF_Chem\202309\wrfout_hourly_d01_2023-09-01_00_00_00.nc
https://data.rda.ucar.edu/ds340.0/202308/wrfout_hourly_d01_2023-08-31_23:00:00.nc
File successfully written.
Saved at: I:\Research_Data\Wild_fire\WRF_Chem\202308\wrfout_hourly_d01_2023-08-31_23_00_00.nc
https://data.rda.ucar.edu/ds340.0/202308/wrfout_hourly_d01_2023-08-31_22:00:00.nc
File successfully written.
Saved at: I:\Research_Data\Wild_fire\WRF_Chem\202308\wrfout_hourly_d01_2023-08-31_22_00_00.nc
https://data.rda.ucar.edu/ds340.0/202308/wrfout_hourly_d01_2023-08-31_21:00:00.nc
File successfully written.
Saved at: I:\Research_Data\Wild_fire\WRF_Chem\202308\wrfout_hourly_d01_2023-08-31_21_00_00.nc
https://data.rda.ucar.edu/ds340.0/202308/wrfout_hourly_d01_2023-08-31_20:00:00.nc
File successfully written.
Saved at: I:\Research_Data\Wild_fire\WRF_Chem\202308\wrfout_hourly_d01_2023-08-

In [ ]:
STOP

In [6]:
files = []

for day in range(1, 31):    
    
    if day < 10:        
        str_day = str(0)+str(day)        
    else:
        str_day = str(day)
    
    for hour in range(24):
        if hour < 10:
            str_hour = str(0)+str(hour)
        else:
            str_hour = str(hour)
        
        file = r"202306/wrfout_hourly_d01_2023-06-"+str_day+"_"+str_hour+":00:00.nc"
        
        files.append(file)

for month in range(7, 9):
    
    for day in range(1, 32):    

        if day < 10:        
            str_day = str(0)+str(day)        
        else:
            str_day = str(day)

        for hour in range(24):
            if hour < 10:
                str_hour = str(0)+str(hour)
            else:
                str_hour = str(hour)

            file = "20230"+str(month)+r"/wrfout_hourly_d01_2023-0"+str(month)+"-"+str_day+"_"+str_hour+":00:00.nc"

            files.append(file)

In [16]:
files[0]

'202306/wrfout_hourly_d01_2023-06-01_00:00:00.nc'

202306/wrfout_hourly_d01_2023-06-01_00:00:00.nc
url:  https://data.rda.ucar.edu/ds340.0/202306/wrfout_hourly_d01_2023-06-01_00:00:00.nc
valid_filename: 202306/wrfout_hourly_d01_2023-06-01_00_00_00.nc
202306/wrfout_hourly_d01_2023-06-01_00_00_00.nc


KeyboardInterrupt: 